## Examples of `mystreams` module usage 

This notebook contains various examples, which demonstrate the capabilities of `mystreams` module: streaming toolkit for PySDK.

This script works with the following inference options:

1. [DeGirum Cloud Platform](https://cs.degirum.com),
1. DeGirum-hosted AI server node shared via Peer-to-Peer VPN,
1. AI server node hosted by you in your local network,
1. AI server running on your local machine,
1. DeGirum ORCA accelerator directly installed on your local machine.

To try different options, you just need to change the `inference_option` in the code below. 

The script also uses either a web camera or local camera connected to the machine running this code. The camera index or URL needs to be specified either in the code below by assigning camera_id or in .env file by defining CAMERA_ID variable and assigning camera_id = None.

In [ ]:
inference_option = 1  # <<< change it according to your needs selecting from the list in the header comment
camera_id = 0         # camera index or URL; 0 to use default local camera, None to take from .env file

### Simplest example: video source is connected to video display

In [ ]:
from mystreams import *

# create Composition: an object, which keeps all streaming blocks (called gizmos) and runs them in parallel
c = Composition();

# create gizmos:
source = VideoSourceGizmo(camera_id) # video source gizmo
display = VideoDisplayGizmo() # video display gizmo

# Create pipeline: connect display input to source output
display.connect_to(source)

# add gizmos to composition
c.add(source)
c.add(display)

# start composition
c.start()

# call c.stop() to stop composition or just press `x` or `q` in display window

### Same example, but with compact syntax

In [ ]:
from mystreams import *

c = Composition();

# Create gizmos and pipeline as a single-liner:
# we use __call__() operator of Composition class instead of add() method
# and we use `>>` operator of gizmo classes instead of connect_to() method
c(VideoSourceGizmo(camera_id)) >> c(VideoDisplayGizmo())

c.start()

### Example of forked streams

In [ ]:
from mystreams import *

c = Composition();

# create and add to composition all required gizmos
source = c.add(VideoSourceGizmo(camera_id))
original_display = c.add(VideoDisplayGizmo("Original"))
resized_display = c.add(VideoDisplayGizmo("Resized"))
resizer = c.add(ResizingGizmo(300, 200))

# Create pipeline: the image source is fed to a display and to the image resizing gizmo, which is then fed to another display.
# It is equivalent to:
#    original_display.connect_to(source)
#    resizer.connect_to(source)
#    resized_display.connect_to(resizer)

source >> original_display
source >> resizer >> resized_display

c.start()

### Example of capturing video input into file with simultaneous display on a screen

In [ ]:
from mystreams import *

c = Composition();

source = c.add(VideoSourceGizmo(camera_id))
display = c.add(VideoDisplayGizmo())
saver = c.add(VideoSaverGizmo("mycapture.mp4"))

source >> display
source >> saver

c.start()

### Example of AI inference from the camera with AI inference results display

In [ ]:
import degirum as dg
import mytools
from mystreams import *

# connect to model zoo according to selected inference option
zoo = mytools.connect_model_zoo(inference_option)

# load some object detection AI model
model = zoo.load_model("yolo_v5s_coco--512x512_quant_n2x_orca_1")

c = Composition();

# create gizmos
source = c.add(VideoSourceGizmo(camera_id)) # video source
detection = c.add(AiSimpleGizmo(model)) # AI model
display = c.add(VideoDisplayGizmo("Detection", show_ai_overlay=True, show_fps=True)) # display

# create pipeline
source >> detection >> display

c.start()

### Example of AI inference from the camera with face detection and mask detection within each detected face bounding box

In [ ]:
import degirum as dg
import mytools
from mystreams import *

# connect to model zoo according to selected inference option
zoo = mytools.connect_model_zoo(inference_option)

# load AI models
face_model = zoo.load_model("yolo_v5s_face_det--512x512_quant_n2x_orca_1")
mask_model = zoo.load_model("mobilenet_v2_mask_yn_cls--224x224_float_n2x_orca_1")
mask_model.overlay_show_probabilities = True

class MaskDetectionGizmo(AiGizmoBase):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._cur_result = None
        
    def on_result(self, result):
        
        meta = result.info[1]
        if "original_result" in meta: # new result
            if self._cur_result is not None:
                # send previous result
                self.send_result((self._cur_result.image, self._cur_result))                
            self._cur_result = meta["original_result"]
        
        if "cropped_index" in meta:
            # apply mask presence/absence label
            if len(result.results) > 0:
                self._cur_result.results[meta["cropped_index"]]["label"] = result.results[0]["label"]

c = Composition();

# create gizmos
source = c.add(VideoSourceGizmo(camera_id)) # video source
face_detection = c.add(AiObjectDetectionCroppingGizmo(face_model)) # face detection gizmo, which outputs cropped images for all detected faces
mask_detection = c.add(MaskDetectionGizmo(mask_model)) # mask detection AI model
face_display = c.add(VideoDisplayGizmo("Faces", show_ai_overlay=True, show_fps=True)) # display

# create pipeline
source >> face_detection >> mask_detection >> face_display

c.start()
